In [8]:

import redis

r = redis.Redis(host='localhost', port=6379, db=0)

In [9]:
from counter import get_last_counter

bank_id = f"bank_{get_last_counter()}"
# bank_id = "bank_52"
bank_id

'bank_58'

In [10]:
from json import loads
from IPython.display import display

index = loads(r.get(f"faiss_index:v1::{bank_id}"))
# Write index to JSON file
import json
with open('faiss_index.json', 'w') as f:
    json.dump(index, f, indent=2)
display(index.keys())
display(len(index['id_by_index']))

dict_keys(['id_by_index', 'chunk_by_index', 'faiss_index'])

8258

In [11]:
import os
from llama_stack_client import LlamaStackClient

# Test run
client = LlamaStackClient(
    base_url="http://localhost:5001",
)

response = client.memory.query(
    bank_id=bank_id,
    query=["What is William Falder's occupation?"],
    params={"max_chunks": 10},
)

display(response)

for chunk, score in zip(response.chunks, response.scores):
    print(chunk)
    print(score)

QueryDocumentsResponse(chunks=[Chunk(content=' 28.284a8 8 0 1 0 11.314 11.314l28.284-28.284a40 40 0 0 1 56.568 56.567l-28.285 28.285a8 8 0 0 0 11.315 11.314l28.284-28.284a56.065 56.065 0 0 0 0-79.196z" fill="currentColor"></path></svg></span></a> <span><strong>This</strong> model is a PyTorch [torch.nn.Module](https —//pytorch.org/docs/stable/nn.html#torch.nn.Module) sub-class. Use</span></span> </li><li class="text-base !pl-4 my-3 rounded "><span class="group flex space-x-1.5 items-start"><a id="transformers.UperNetForSemanticSegmentation.it" class="header-link block pr-0.5 text-lg no-hover:hidden with-hover:absolute with-hover:p-1.5 with-hover:opacity-0 with-hover:group-hover:opacity-100 with-hover:right-full" href="#transformers.UperNetForSemanticSegmentation.it"><span><svg class="text-smd" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" aria-hidden="true" role="img" width="1em" height="1em" preserveAspectRatio="xMidYMid meet" viewBox="0 0 256 256"><pat

Chunk(content=' 28.284a8 8 0 1 0 11.314 11.314l28.284-28.284a40 40 0 0 1 56.568 56.567l-28.285 28.285a8 8 0 0 0 11.315 11.314l28.284-28.284a56.065 56.065 0 0 0 0-79.196z" fill="currentColor"></path></svg></span></a> <span><strong>This</strong> model is a PyTorch [torch.nn.Module](https —//pytorch.org/docs/stable/nn.html#torch.nn.Module) sub-class. Use</span></span> </li><li class="text-base !pl-4 my-3 rounded "><span class="group flex space-x-1.5 items-start"><a id="transformers.UperNetForSemanticSegmentation.it" class="header-link block pr-0.5 text-lg no-hover:hidden with-hover:absolute with-hover:p-1.5 with-hover:opacity-0 with-hover:group-hover:opacity-100 with-hover:right-full" href="#transformers.UperNetForSemanticSegmentation.it"><span><svg class="text-smd" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" aria-hidden="true" role="img" width="1em" height="1em" preserveAspectRatio="xMidYMid meet" viewBox="0 0 256 256"><path d="M167.594 88.393a8.001 8.00

In [12]:
import pandas as pd
import datasets

# dataset = datasets.load_dataset(path="data/transformers.parquet")

# df = dataset["train"].to_pandas()
df = pd.read_parquet("data/transformers.parquet")
df = df.rename(columns={'html': 'context'})
df['context_length'] = df['context'].apply(lambda x: len(x))
df.describe()

,context_length
count,50.00000
mean,178385.28000
std,92226.00916
min,253.00000
25%,159669.25000
50%,209115.50000
75%,240418.50000
max,296460.00000


In [13]:
doc_ids = []
titles = []
for i, row in df.iterrows():
    titles.append(row["title"])
    doc_ids.append(i)

print(len(titles))
print(len(doc_ids))
display(titles[:5])
doc_ids[:40]

50
50


['',
 'Translation',
 'Run training on Amazon SageMaker',
 'Create a custom architecture',
 'Image Processor']

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39]

In [14]:
from tqdm import tqdm

top_k = 10

results = []

for i in tqdm(range(0, len(titles))):
    response = client.memory.query(
        bank_id=bank_id,
        query=[titles[i]],
        params={"max_chunks": top_k},
    )
    res = [int(chunk.document_id) for chunk in response.chunks]
    results.append(res)

results

 38%|████████████████████████████▌                                              | 19/50 [00:00<00:00, 90.39it/s]

100%|███████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 89.10it/s]


[[25, 4, 25, 28, 33, 41, 36, 42, 4, 41],
 [28, 28, 1, 24, 1, 24, 1, 1, 41, 36],
 [22, 6, 1, 2, 12, 2, 42, 5, 48, 44],
 [3, 3, 22, 22, 26, 29, 3, 49, 3, 11],
 [4, 18, 3, 26, 16, 35, 42, 42, 28, 44],
 [5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
 [4, 45, 1, 35, 16, 6, 7, 32, 26, 5],
 [7, 7, 5, 7, 7, 7, 7, 17, 7, 7],
 [8, 8, 8, 8, 4, 8, 8, 31, 8, 8],
 [25, 4, 25, 28, 33, 41, 36, 42, 4, 41],
 [10, 17, 33, 10, 10, 7, 5, 17, 10, 1],
 [11, 31, 11, 11, 11, 17, 1, 7, 2, 5],
 [12, 12, 12, 12, 12, 28, 12, 28, 6, 6],
 [25, 4, 25, 28, 33, 41, 36, 42, 4, 41],
 [41, 3, 14, 24, 26, 41, 40, 14, 17, 17],
 [18, 48, 15, 44, 44, 44, 15, 44, 44, 44],
 [16, 16, 16, 16, 16, 16, 16, 16, 16, 16],
 [17, 7, 5, 17, 42, 12, 20, 49, 17, 23],
 [48, 18, 38, 11, 44, 48, 44, 3, 25, 44],
 [19, 19, 42, 19, 19, 19, 19, 19, 19, 19],
 [20, 4, 15, 26, 33, 15, 32, 19, 36, 35],
 [25, 4, 25, 28, 33, 41, 36, 42, 4, 41],
 [22, 32, 49, 22, 14, 24, 24, 22, 49, 22],
 [14, 35, 23, 26, 41, 25, 38, 15, 17, 23],
 [22, 24, 4, 1, 6, 5, 7, 33, 19, 28],


In [23]:
import numpy as np

top_k = 3

res = np.array(results)


targets = np.array(doc_ids)
display(targets.shape)
targets = np.expand_dims(targets, axis=1)
display(targets.shape)
targets = targets == res
display(targets.shape)
targets = targets.astype(float)
display(targets)

targets = targets[:, :top_k]
display(targets.shape)

(50,)

(50, 1)

(50, 10)

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 1., 0., 1., 1., 0., 0.],
       [0., 0., 0., 1., 0., 1., 0., 0., 0., 0.],
       [1., 1., 0., 0., 0., 0., 1., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 1., 0., 1., 1., 1., 1., 0., 1., 1.],
       [1., 1., 1., 1., 0., 1., 1., 0., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 1., 1., 0., 0., 0., 1., 0.],
       [1., 0., 1., 1., 1., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0., 1., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 0., 0., 1., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 0., 1., 1., 1., 1., 1., 1., 1.],
       [1., 0., 0., 

(50, 3)

In [24]:
temp = np.sum(targets, axis=1)
temp = np.divide(temp, top_k)
precision = np.sum(temp) / len(temp)
precision

np.float64(0.4666666666666666)

In [25]:
temp = np.logical_or.reduce(targets, axis=1)
temp = temp.astype(float)
recall = np.sum(temp) / len(temp)
recall

np.float64(0.7)

In [26]:
ranks = np.tile(np.arange(1, top_k + 1), (len(titles), 1)).astype(float)
temp = np.copy(targets)
for i in range(len(temp)):
    first_true = True
    for j in range(len(temp[i])):
        if temp[i,j]:
            if first_true:
                first_true = False
            else:
                temp[i,j] = False
reciprocal_ranks = temp / ranks
reciprocal_ranks = np.sum(reciprocal_ranks) / len(temp)
reciprocal_ranks = np.mean(reciprocal_ranks)
reciprocal_ranks

np.float64(0.6166666666666667)